<a href="https://colab.research.google.com/github/GabrielaCuervoR/Analitica3/blob/main/b_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from platform import python_version ## versión de python
import pandas as pd ### para manejo de datos
import sqlite3 as sql #### para bases de datos sql
import a_funciones as funciones  ###archivo de funciones propias
import sys ## saber ruta de la que carga paquetes
from sklearn.preprocessing import LabelEncoder ### Transformar la variable objetivo en numérica.

In [ ]:
##Se cargan las bases de datos almacenadas en el github
df_data= 'https://raw.githubusercontent.com/GabrielaCuervoR/Analitica3/main/Data/general_data.csv'
df_retiros= 'https://raw.githubusercontent.com/GabrielaCuervoR/Analitica3/main/Data/retirement_info.csv'
df_employee= 'https://raw.githubusercontent.com/GabrielaCuervoR/Analitica3/main/Data/employee_survey_data.csv'
df_manager= 'https://raw.githubusercontent.com/GabrielaCuervoR/Analitica3/main/Data/manager_survey_data.csv'

In [ ]:
##Se leen la base de datos con la información arreglada
df_data=pd.read_csv(df_data, sep= ';' )
df_retiros=pd.read_csv(df_retiros, sep=';')
df_employee=pd.read_csv(df_employee)
df_manager=pd.read_csv(df_manager)

In [ ]:
conn= sql.connect("db_empleados") ## Con este comando se crea la base de datos o si ya existe se conecta

df_employee.to_sql("employee_survey_data",conn,if_exists="replace")
df_data.to_sql("general_data",conn,if_exists="replace")
df_manager.to_sql("manager_survey_data",conn,if_exists="replace")
df_retiros.to_sql("retirement_info",conn,if_exists="replace")

711

In [ ]:
cur= conn.cursor()
funciones.ejecutar_sql('preprocesamiento.sql', cur)

In [ ]:
df2=pd.read_sql("select * from base_nueva", conn)

In [ ]:
df2 = df2.drop(columns=['index'], axis=1)##Se elimina la columna index que por defecto la sube sql

In [ ]:
df2.info()##Se analiza la información de la base de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   BusinessTravel           4410 non-null   object 
 2   Department               4410 non-null   object 
 3   DistanceFromHome         4410 non-null   int64  
 4   Education                4410 non-null   int64  
 5   EducationField           4410 non-null   object 
 6   EmployeeCount            4410 non-null   int64  
 7   EmployeeID               4410 non-null   int64  
 8   Gender                   4410 non-null   object 
 9   JobLevel                 4410 non-null   int64  
 10  JobRole                  4410 non-null   object 
 11  MaritalStatus            4410 non-null   object 
 12  MonthlyIncome            4410 non-null   int64  
 13  NumCompaniesWorked       4391 non-null   float64
 14  Over18                  

In [ ]:
### Se verifican las categorias y obsevaciones de cada variable categorica, para ver si se deben agrupar por si tiene muchas categorias.
pd.read_sql("""select BusinessTravel,count(*)
                            from base_nueva
                            group by BusinessTravel""", conn)

,BusinessTravel,count(*)
0,Non-Travel,450
1,Travel_Frequently,831
2,Travel_Rarely,3129


In [ ]:
pd.read_sql("""select Department,count(*)
                            from base_nueva
                            group by Department""", conn)

,Department,count(*)
0,Human Resources,189
1,Research & Development,2883
2,Sales,1338


In [ ]:
pd.read_sql("""select EducationField ,count(*)
                            from base_nueva
                            group by EducationField """, conn)

,EducationField,count(*)
0,Human Resources,81
1,Life Sciences,1818
2,Marketing,477
3,Medical,1392
4,Other,246
5,Technical Degree,396


In [ ]:
pd.read_sql("""select Gender ,count(*)
                            from base_nueva
                            group by Gender""", conn)

,Gender,count(*)
0,Female,1764
1,Male,2646


In [ ]:
##Esta variable es la que tiene más categorias pero lo que representa son el cargo de cada representante por lo tanto se procede a dejarla asi.
pd.read_sql("""select  JobRole ,count(*)
                            from base_nueva
                            group by JobRole """, conn)

,JobRole,count(*)
0,Healthcare Representative,393
1,Human Resources,156
2,Laboratory Technician,777
3,Manager,306
4,Manufacturing Director,435
5,Research Director,240
6,Research Scientist,876
7,Sales Executive,978
8,Sales Representative,249


In [ ]:
pd.read_sql("""select MaritalStatus ,count(*)
                            from base_nueva
                            group by MaritalStatus """, conn)

,MaritalStatus,count(*)
0,Divorced,981
1,Married,2019
2,Single,1410


In [ ]:
##Esta variable tiene una sola categoria por lo que no aportaria al modelo y se debe eliminar.
pd.read_sql("""select  Over18 ,count(*)
                            from base_nueva
                            group by Over18 """, conn)

,Over18,count(*)
0,Y,4410


In [ ]:
##Esta variable en nuestra variable objetivo por lo tanto se debe convertir a numerica para que el modelo funcione mejor.
pd.read_sql("""select  Attrition ,count(*)
                            from base_nueva
                            group by Attrition """, conn)

,Attrition,count(*)
0,None,3699
1,Yes,711


In [ ]:
df2.isnull().sum()

Age                           0
BusinessTravel                0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
EmployeeCount                 0
EmployeeID                    0
Gender                        0
JobLevel                      0
JobRole                       0
MaritalStatus                 0
MonthlyIncome                 0
NumCompaniesWorked           19
Over18                        0
PercentSalaryHike             0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             9
TrainingTimesLastYear         0
YearsAtCompany                0
YearsSinceLastPromotion       0
YearsWithCurrManager          0
Attrition                  3699
JobInvolvement                0
PerformanceRating             0
EnvironmentSatisfaction      25
WorkLifeBalance              38
JobSatisfaction              20
dtype: int64

In [ ]:
##Se observa cuanto nulos tiene cada variable para más adelante proceder a la imputación (Esto se va a hacer en el archivo donde se crea el modelo).
##El análisis de las variables numéricas se va a realizar en la parte de exploración de datos.
##Se decide que las variables categóricas se van a rellenar con la moda que es el valor que más se repite.
##La variable Attrition se reemplazan los nulos por la palabra No. Ya que nuestra variable objetivo nos va a decir si un empleado se fue o no y también se observa que esta un poco sesgada los datos entonces hay que hacerle una estandarización a la variable.
##La variable numérica se van a rellenar con la mediana porque es una medida  que no se ve afectada por valores atípicos de los datos.